
# 가중치만 저장해서 모델 용량 줄이기

In [6]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import os

bath_path = "/104data/models/spick_used_models/" #완성된 모델 경로

for model_folder_name in ["MA", "NA", "NAB", "NM"]:
    folder_path = os.path.join(bath_path, model_folder_name)
    model_files = [f for f in os.listdir(folder_path) if f.endswith('.h5')]

    for model_file in model_files:
        model_path = os.path.join(folder_path, model_file)
        # Load the model
        model = tf.keras.models.load_model(model_path)
        
        # Modify the name to save weights only
        weight_file_name = model_file.replace('model', 'weight')
        weight_file_path = os.path.join(folder_path, weight_file_name)
        
        # Save weights
        model.save_weights(weight_file_path)

        print(f"Saved weights: {weight_file_path}")


2024-12-05 18:19:16.606086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-05 18:19:16.637786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-05 18:19:16.637917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-05 18:19:16.638438: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_10_67.1.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_05_63.4.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_03_64.6.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_02_64.6.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_04_67.1.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_08_69.5.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_07_64.6.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_06_62.2.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_09_69.5.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_11_67.1.h5
Saved weights: /104data/models/spick_used_models/MA/save_weight_MvsA_01_68.3.h5
Saved weights: /104data/models/spick_used_models/NA/save_weight_SvsA_06_72.0.h5
Saved weights: /104data/models/spick_use

# 양자화를 통한 모델 줄이기
tensorflow는 양자화를 진행하게되면 predict 하는 방식에 변화가 많이 생김. 양자화 후에는 모델 재훈련 불가 원본 모델은 형상관리를 위해 남겨둘것

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import os

# 완성된 모델 경로
bath_path = "/104data/models/spick_used_models/" 

# 양자화 및 변환을 진행할 폴더들
for model_folder_name in ["MA", "NA", "NAB", "NM"]:
    folder_path = os.path.join(bath_path, model_folder_name)
    model_files = [f for f in os.listdir(folder_path) if f.endswith('.h5') and "weight" not in f]


    for model_file in model_files:
        model_path = os.path.join(folder_path, model_file)

        # Load the model
        model = tf.keras.models.load_model(model_path)
        
        # TFLite 변환기 생성
        converter = tf.lite.TFLiteConverter.from_keras_model(model)

        # 기본 양자화 (float32 -> float16 가중치로 줄임)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]

        # 양자화 모델 변환
        tflite_model = converter.convert()

        # 양자화된 파일 저장 경로 생성
        tflite_file_name = model_file.replace('.h5', '_quantized.tflite')
        tflite_file_path = os.path.join(folder_path, tflite_file_name)

        # 양자화 모델 저장
        with open(tflite_file_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"Saved quantized model: {tflite_file_path}")


INFO:tensorflow:Assets written to: /tmp/tmpk225gxaj/assets


INFO:tensorflow:Assets written to: /tmp/tmpk225gxaj/assets
2024-12-06 14:57:38.926398: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2024-12-06 14:57:38.926420: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2024-12-06 14:57:38.926569: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpk225gxaj
2024-12-06 14:57:38.929061: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2024-12-06 14:57:38.929077: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpk225gxaj
2024-12-06 14:57:38.938404: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2024-12-06 14:57:39.049533: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmpk225gxaj
2024-12-06 14:57:39.071007: I tensorflow/cc/saved_model/loader.cc:283] SavedMode

Saved quantized model: /104data/models/spick_used_models/MA/save_model_MvsA_10_67.1_quantized.tflite
INFO:tensorflow:Assets written to: /tmp/tmp_0953hn3/assets


INFO:tensorflow:Assets written to: /tmp/tmp_0953hn3/assets
2024-12-06 14:58:32.098250: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2024-12-06 14:58:32.098276: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2024-12-06 14:58:32.098435: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp_0953hn3
2024-12-06 14:58:32.100869: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2024-12-06 14:58:32.100885: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmp_0953hn3
2024-12-06 14:58:32.110507: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2024-12-06 14:58:32.207478: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmp_0953hn3
2024-12-06 14:58:32.229305: I tensorflow/cc/saved_model/loader.cc:283] SavedMode

Saved quantized model: /104data/models/spick_used_models/MA/save_model_MvsA_05_63.4_quantized.tflite
INFO:tensorflow:Assets written to: /tmp/tmputistkw3/assets


INFO:tensorflow:Assets written to: /tmp/tmputistkw3/assets
2024-12-06 14:59:14.675597: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2024-12-06 14:59:14.675620: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2024-12-06 14:59:14.675761: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmputistkw3
2024-12-06 14:59:14.678099: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2024-12-06 14:59:14.678113: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmputistkw3
2024-12-06 14:59:14.687573: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2024-12-06 14:59:14.789692: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmputistkw3
2024-12-06 14:59:14.810999: I tensorflow/cc/saved_model/loader.cc:283] SavedMode

Saved quantized model: /104data/models/spick_used_models/MA/save_model_MvsA_03_64.6_quantized.tflite
INFO:tensorflow:Assets written to: /tmp/tmp8tuh3k2l/assets


INFO:tensorflow:Assets written to: /tmp/tmp8tuh3k2l/assets
2024-12-06 14:59:56.510483: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2024-12-06 14:59:56.510509: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2024-12-06 14:59:56.510666: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp8tuh3k2l
2024-12-06 14:59:56.512873: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2024-12-06 14:59:56.512889: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmp8tuh3k2l
2024-12-06 14:59:56.521720: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2024-12-06 14:59:56.637345: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmp8tuh3k2l
2024-12-06 14:59:56.657701: I tensorflow/cc/saved_model/loader.cc:283] SavedMode

Saved quantized model: /104data/models/spick_used_models/MA/save_model_MvsA_02_64.6_quantized.tflite
